# Store kitti sequence

Create a sequence

In [1]:
# when you change a setting you have to restart the kernel (settings are only loaded once)
import os
#os.environ['STEREO_CONFIDENCE'] = '1.6'
#os.environ['SEQUENCE_CONFIDENCE'] = '1.6'
#os.environ['PATCH_SIZE'] = '17'
FRAMECOUNT = 1101

import sys
sys.path.append('..')
from src.kitti import *
%matplotlib inline
import numpy as np
import glob
import urbg2o
import cv2


IMAGE_SIZE = 400

def load(file):
    keyframeids, frameids, poses = load_keyframes(file)
    return keyframeids, frameids, poses.reshape(poses.shape[0], 4, 4)

def load_csv(file):
    poses = np.genfromtxt(file, delimiter= " ")
    poses = np.hstack([poses, np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) ])
    return poses.reshape(poses.shape[0], 4, 4)

def transform(poses):
    points = np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) 
    for i in range(1, len(poses)):
        points[:i] = np.dot(points[:i], poses[i].T)
    return points
        
def transform2(poses):
    points = poses[:,:,3]
    return points
    
def plot_trajectory(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1])

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100

    return np.dot(points, t.T)
    

def plot_trajectory2(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1]) * 1.01

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100
    return np.dot(points, t.T)
    
def plot_trajectory3(points):
    coords = []
    minx = min([p[0] for p in points])
    maxx = max([p[0] for p in points])
    miny = min([p[2] for p in points])
    maxy = max([p[2] for p in points])
    maxxy = max(maxx - minx, maxy - miny)* 1.01

    for p in points:
        #print(p)
        x = IMAGE_SIZE * (p[0] - minx) / (maxxy)
        z = IMAGE_SIZE - IMAGE_SIZE * (p[2] - miny) / (maxxy)
        coords.append([x, z])
    return coords

def new_image():
    img = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), np.uint8)
    img[:] = 255 #or img.fill(255)
    return img

def draw(coords, img, color = (0,0,0)):
    # Draw a black line with thickness of 2 px
    for i in range(1,len(coords)):
        cv2.line(img, (int(coords[i - 1][0]), int(coords[i - 1][1])), (int(coords[i][0]), int(coords[i][1])), color, 2)
    return img

Read the first 10 frames from KITTI sequence 00.

In [2]:
%%time
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/00/image_0'
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/00/image_1'

frame = []
for filename in sorted(glob.glob(LEFTDIR + '/*')):
    left_frame = Frame(filename, RIGHTDIR)
    frame.append(left_frame)

CPU times: user 16.1 ms, sys: 4.36 ms, total: 20.4 ms
Wall time: 19.3 ms


Add the frames to a sequence. By adjusting the [:] you can use only a slice of the list of frames.

In [3]:
seq = Sequence()
for f in ProgressBar()(frame[:10]):
    seq.add_keyframe(f)

N/A% (0 of 10) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

set_previous_keyframe 0


 10% (1 of 10) |##                        | Elapsed Time: 0:00:03 ETA:  0:00:28

get_world_pose first  0
set_previous_keyframe 1


 20% (2 of 10) |#####                     | Elapsed Time: 0:00:07 ETA:  0:00:31

...

get_world_pose  7 [[  9.99267799e-01  -3.85543517e-05   3.82604936e-02  -2.10637330e-01]
 [ -1.17481041e-04   9.99991686e-01   4.07598120e-03   7.19808697e-02]
 [ -3.82603326e-02  -4.07749164e-03   9.99259486e-01  -5.24187944e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
set_previous_keyframe 8


 90% (9 of 10) |#######################   | Elapsed Time: 0:00:40 ETA:  0:00:04

get_world_pose  8 [[  9.99033995e-01  -6.61531321e-04   4.39390326e-02  -2.57535960e-01]
 [  5.75188633e-04   9.99997879e-01   1.97767030e-03   7.80220597e-02]
 [ -4.39402477e-02  -1.95048663e-03   9.99032257e-01  -6.05330803e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
set_previous_keyframe 9


100% (10 of 10) |#########################| Elapsed Time: 0:00:44 Time: 0:00:44


get_world_pose  9 [[  9.98605050e-01  -4.43153272e-03   5.26147835e-02  -5.08642685e-01]
 [  4.61097433e-03   9.99983959e-01  -3.28958151e-03   1.54724419e-01]
 [ -5.25993616e-02   3.52759812e-03   9.98609465e-01  -6.50052003e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


Example of how to save a set of covisible keyframes, fixed keyframes, mappoints and edges to use for local bundle adjustment

In [4]:
len(frame[5].get_observations())

1124

In [5]:
mp1 = [o.mappoint for o in frame[1].get_observations() if o.mappoint is not None]
[len(m.observations) for m in mp1]

[1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 3,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 3,
 4,
 1,
 2,
 6,
 1,
 1,
 3,
 4,
 4,
 2,
 2,
 1,
 3,
 3,
 1,
 4,
 2,
 3,
 2,
 1,
 1,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 2,
 10,
 1,
 2,
 3,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 3,
 2,
 3,
 4,
 5,
 4,
 2,
 1,
 3,
 1,
 3,
 9,
 4,
 2,
 2,
 2,
 5,
 1,
 1,
 2,
 3,
 4,
 1,
 6,
 2,
 5,
 1,
 1,
 2,
 3,
 4,
 2,
 10,
 2,
 7,
 2,
 4,
 5,
 1,
 2,
 1,
 3,
 3,
 3,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 4,
 1,
 4,
 2,
 2,
 4,
 3,
 10,
 1,
 1,
 4,
 1,
 4,
 3,
 2,
 2,
 1,
 1,
 5,
 2,
 2,
 4,
 3,
 2,
 1,
 2,
 2,
 3,
 2,
 3,
 4,
 2,
 4,
 1,
 3,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 5,
 2,
 3,
 2,
 1,
 2,
 3,
 2,
 6,
 1,
 2,
 5,
 3,
 10,
 2,
 2,
 3,
 2,
 1,
 2,
 2,
 1,
 10,
 5,
 2,
 4,
 3,
 4,
 2,
 3,
 3,
 2,
 9,
 1,
 5,
 1,
 1,
 2,
 2,
 2,
 4,
 5,
 2,
 5,
 1,
 5,
 2,
 1,
 3,
 1,
 2,
 1,
 2,
 1,
 3,
 1,
 1,
 2,
 9,
 2,
 2,
 5,
 1,
 2,
 1,
 2,
 1,
 4,
 1,
 4,
 2,
 2,
 10,
 1

In [6]:

print('Keyframes count\n', (len(seq.keyframes)), '\n')

cv_keyframes = get_covisible_keyframes(seq.keyframes[1])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

# np.set_printoptions(threshold=np.nan)

print('mappoints count  \n', (len(mappoints[:]), '\n') )
print('mappoints \n', (mappoints[:]), '\n')


print('links count  \n', (len(links[:]), '\n') )
print('links \n', (links[:]), '\n')


print('Keyframes \n', (cv_keyframes), '\n')
print('Keyframes numpy \n', (cv_keyframes_np[:, 2 :]), '\n')
print('mappoints len \n', len(mappoints_np), '\n')
print('Fixed Keyframes \n',f_keyframes[:], '\n')
# print( keyframes_to_np( [seq.keyframes[1]]  )[:, 2 :] , '\n')


print('Keyframes before BA \n', (cv_keyframes_BA[: , 2:]), '\n')
print(len(cv_keyframes_BA), '\n')

result =  urbg2o.localBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

print('Keyframes after BA \n', (cv_keyframes_BA[:, 2 :]), '\n')
print(len(cv_keyframes_BA), '\n')




# print(cv_keyframes[:, 0 :], '\n')

np_matrix_values =  cv_keyframes_np[:,2 :]


# draw seq.keyframes

# array_value = []
# for i in range(0, len(keyframes_to_np(seq.keyframes)[:,2 :]) ):
#     array_value.append(  np.split(keyframes_to_np(seq.keyframes)[:,2 :][i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)



#old points 
# array_value = []
# for i in range(0, len(np_matrix_values) ):
#     array_value.append(  np.split(np_matrix_values[i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)

# #new points

# start_array_value = []
# for i in range(0, len(cv_keyframes_BA[:,2 :] ) ):
#     start_array_value.append(  np.split(cv_keyframes_BA[:,2 :][i], 4 )  )
# start_array_value = np.array(start_array_value)

# points_start = transform( start_array_value )
# coords_start = plot_trajectory3(points_start)
# p_start = draw(coords_start, new_image())
# show(p_start)


keyframes_np_MO = keyframes_to_np(seq.keyframes)

# np.save('cv_keyframes', cv_keyframes_BA)
# np.save('f_keyframes', f_keyframes_np)
# np.save('mappoints', mappoints_np)
# np.save('links', links_np)


Keyframes count
 10 

mappoints count  
 (2255, '\n')
mappoints 
 [[  2.76000000e+03  -5.97239311e-02   1.71355868e+00   7.01923192e+00
    1.00000000e+00]
 [  3.12300000e+03   9.24783320e-01  -3.33570548e+00   1.53415012e+01
    1.00000000e+00]
 [  3.12500000e+03   1.25993456e+01  -2.92500250e+00   1.37862652e+01
    1.00000000e+00]
 ..., 
 [  3.25800000e+03   2.62423993e+00   6.69894058e-01  -1.15867874e-02
    1.00000000e+00]
 [  3.26000000e+02  -1.56450115e+01  -3.87756988e+00   9.92685610e+01
    1.00000000e+00]
 [  7.35000000e+02  -2.44132639e+00   9.18480779e-01   2.46508372e+01
    1.00000000e+00]] 

links count  
 (6526, '\n')
links 
 [[  2.76000000e+03   4.00000000e+00   5.93000000e+02   3.06000000e+02]
 [  2.76000000e+03   3.00000000e+00   5.90000000e+02   2.97000000e+02]
 [  3.12300000e+03   5.00000000e+00   6.28000000e+02   5.50000000e+01]
 ..., 
 [  3.26000000e+02   3.00000000e+00   4.80000000e+02   1.53000000e+02]
 [  7.35000000e+02   0.00000000e+00   5.36000000e+02   2.

In [7]:

print('Keyframes count\n', (len(seq.keyframes)), '\n')

def keyframes_to_np(keyframes):
    return np.hstack([ [ [ kf.keyframeid] for kf in keyframes ], [ [kf.frameid] for kf in keyframes ], [ kf.cumulative_pose.flatten() for kf in keyframes ] ])

for i in range(len(seq.keyframes)):
    f = seq.keyframes[i]
    pose = f.get_pose()
    if i > 0:
        f.cumulative_pose = np.dot(seq.keyframes[i-1].cumulative_pose, pose)
    else:
        f.cumulative_pose = pose

cv_keyframes = get_covisible_keyframes(seq.keyframes[1])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)[:0]

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

# np.set_printoptions(threshold=np.nan)

print('mappoints count  \n', (len(mappoints[:]), '\n') )
print('mappoints \n', (mappoints[:]), '\n')


print('links count  \n', (len(links[:]), '\n') )
print('links \n', (links[:]), '\n')


print('Keyframes \n', (cv_keyframes), '\n')
print('Keyframes numpy \n', (cv_keyframes_np[:, 2 :]), '\n')
print('mappoints len \n', len(mappoints_np), '\n')
print('Fixed Keyframes \n',f_keyframes[:], '\n')
# print( keyframes_to_np( [seq.keyframes[1]]  )[:, 2 :] , '\n')


print('Keyframes before BA \n', (cv_keyframes_BA[: , 2:]), '\n')
print(len(cv_keyframes_BA), '\n')

result =  urbg2o.localBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

print('Keyframes after BA \n', (cv_keyframes_BA[:, 2 :]), '\n')
print(len(cv_keyframes_BA), '\n')




# print(cv_keyframes[:, 0 :], '\n')

np_matrix_values =  cv_keyframes_np[:,2 :]


# draw seq.keyframes

# array_value = []
# for i in range(0, len(keyframes_to_np(seq.keyframes)[:,2 :]) ):
#     array_value.append(  np.split(keyframes_to_np(seq.keyframes)[:,2 :][i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)



#old points 
# array_value = []
# for i in range(0, len(np_matrix_values) ):
#     array_value.append(  np.split(np_matrix_values[i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)

# #new points

# start_array_value = []
# for i in range(0, len(cv_keyframes_BA[:,2 :] ) ):
#     start_array_value.append(  np.split(cv_keyframes_BA[:,2 :][i], 4 )  )
# start_array_value = np.array(start_array_value)

# points_start = transform( start_array_value )
# coords_start = plot_trajectory3(points_start)
# p_start = draw(coords_start, new_image())
# show(p_start)


keyframes_np_MO = keyframes_to_np(seq.keyframes)

# np.save('cv_keyframes', cv_keyframes_BA)
# np.save('f_keyframes', f_keyframes_np)
# np.save('mappoints', mappoints_np)
# np.save('links', links_np)


Keyframes count
 10 

mappoints count  
 (2255, '\n')
mappoints 
 [[  2.76000000e+03  -5.97239311e-02   1.71355868e+00   7.01923192e+00
    1.00000000e+00]
 [  3.12300000e+03   9.24783320e-01  -3.33570548e+00   1.53415012e+01
    1.00000000e+00]
 [  3.12500000e+03   1.25993456e+01  -2.92500250e+00   1.37862652e+01
    1.00000000e+00]
 ..., 
 [  3.25800000e+03   2.62423993e+00   6.69894058e-01  -1.15867874e-02
    1.00000000e+00]
 [  3.26000000e+02  -1.56450115e+01  -3.87756988e+00   9.92685610e+01
    1.00000000e+00]
 [  7.35000000e+02  -2.44132639e+00   9.18480779e-01   2.46508372e+01
    1.00000000e+00]] 

links count  
 (6526, '\n')
links 
 [[  2.76000000e+03   4.00000000e+00   5.93000000e+02   3.06000000e+02]
 [  2.76000000e+03   3.00000000e+00   5.90000000e+02   2.97000000e+02]
 [  3.12300000e+03   5.00000000e+00   6.28000000e+02   5.50000000e+01]
 ..., 
 [  3.26000000e+02   3.00000000e+00   4.80000000e+02   1.53000000e+02]
 [  7.35000000e+02   0.00000000e+00   5.36000000e+02   2.

In [8]:


for i in range(len(seq.keyframes) - 1, 0, -1):
    f = seq.keyframes[i]
    prev = seq.keyframes[i-1]
    previnv = np.linalg.pinv(prev.cumulative_pose)
    pose = np.dot(previnv, f.cumulative_pose)
    print(i, f.get_pose(), '\n', pose)

9 [[ 0.99995427 -0.00400708  0.00868292 -0.23116943]
 [ 0.00405295  0.99997789 -0.00527216  0.07774059]
 [-0.0086616   0.00530711  0.9999484  -0.45766091]
 [ 0.          0.          0.          1.        ]] 
 [[  9.99954274e-01  -4.00707597e-03   8.68291800e-03  -2.31169429e-01]
 [  4.05295039e-03   9.99977889e-01  -5.27215527e-03   7.77405929e-02]
 [ -8.66160008e-03   5.30710563e-03   9.99948404e-01  -4.57660909e-01]
 [  2.80754000e-17  -9.47872470e-15  -8.54337867e-17   1.00000000e+00]]
8 [[  9.99983602e-01  -7.03901471e-04   5.68332160e-03  -1.58194734e-02]
 [  7.15832736e-04   9.99997544e-01  -2.09758586e-03   9.35154119e-03]
 [ -5.68183115e-03   2.10161977e-03   9.99981650e-01  -8.12597448e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]] 
 [[  9.99983602e-01  -7.03901471e-04   5.68332160e-03  -1.58194734e-02]
 [  7.15832736e-04   9.99997544e-01  -2.09758586e-03   9.35154119e-03]
 [ -5.68183115e-03   2.10161977e-03   9.99981650e-01  -8.12597448e-01]
 [  